In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def evaluate_rules(data, tuned_rules, output):
    predictions_all = []
    for i, rule in tuned_rules.iterrows():
        pred_rule = []
        tptot = 0
        tntot = 0
        fntot = 0
        fptot = 0
        tuned_antecedent = rule['Rule'].strip()
        print(f"rule: {tuned_antecedent}")
        for _, row in data.iterrows():
            y_row = row[output]
            tp = 0
            tn = 0
            fn = 0
            fp = 0
            # check if the point row satifies rule 
            if evaluate_rule_conditions(row, tuned_antecedent):
                # rule is satisfied
                satisfied = True
                pred_rule.append(0)
            else:
                satisfied = False
                pred_rule.append(1)
    
            
            if satisfied and y_row == 0:
                tp=1
            elif satisfied and y_row == 1:
                fp=1
            elif not satisfied and y_row == 1:
                tn=1
            
            elif not satisfied and y_row == 0:
                fn=1

            predictions_all.append(pred_rule)
            tptot+=tp
            tntot+=tn
            fptot+=fp
            fntot+=fn

        if (tptot+fptot) == 0:
            print("no points satisfy the rule\n")
            continue
        else:
            
            print(f"tp = {tptot}, tn = {tntot}, fp = {fptot}, fn = {fntot}")
            prec = tptot / (tptot+fptot)
            cov = tptot/(tptot+fntot)
            acc = (tptot+tntot)/(tptot+tntot+fptot+fntot)
            f1score = (2*tptot)/(2*tptot+fptot+fntot)
            error = fptot/(fptot+tntot)
            print(f"precision: {prec}, covering: {cov}, error: {error}, accuracy: {acc}, f1: {f1score}\n")              

    return predictions_all


def evaluate_rule_conditions(row, condition_part):

    # Checks if any of the conditions in the rule are satisfied
    if all(check_condition(row, part) for part in condition_part.split(" and ")):
        return True  # Return True if all conditions in the rule are satisfied
    
    return False  # Return False if any of the conditions in the rule is not satisfied


def check_condition(row, condition_part):
    # Check if a single condition part is satisfied
    
    #parts = [part.strip('()') for part in condition_part.split()]
    parts = re.split(r'\s*(==|<=|>=|<|>|!=)\s*', condition_part.strip('()'))
    #print("parts: ", parts)
    if len(parts) == 3:
        column, op, value = parts
        return eval(f"{row[column]} {op} {value}")
    # handle the case of a 2-thresholds conditions of the kind: a < Column <= b
    elif len(parts) == 5:
        val1,op1,column,op2,val2 = parts
        # Use the original condition from the rule
        return eval(f"{val1} {op1} {row[column]} {op2} {val2}")        
    else:
        # Handle cases where the condition is not in the expected format
        pass


In [3]:
# Loading rules actual rules
rules = pd.read_csv('/Users/saranarteni/OneDrive - CNR/REXASI-PRO/T3.1/navground_code/simulation2/confiderai/supsinavground_rules.csv', header=None, names=["Rule", "Covering", "Error"])

# adjust columns values
for i in range(len(rules)+1):   
    rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("AND","and"))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("{",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("}",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("output in ","output = "))
rules["Covering"] = rules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
rules["Error"] = rules["Error"].apply(lambda x: x.replace("ERROR: ",""))
rules['Output'] = rules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("IF ",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))
# load test data
data = pd.read_csv("simulation2/test.csv")

rules_noncoll = rules[rules["Output"]==0]
preds_LLM = evaluate_rules(data, rules_noncoll, "output")

rule: SafetyMargin > 0.071752 and Tau <= 0.798864
tp = 219, tn = 431, fp = 18, fn = 332
precision: 0.9240506329113924, covering: 0.397459165154265, error: 0.0400890868596882, accuracy: 0.65, f1: 0.5558375634517766

rule: SafetyMargin > 0.019271 and 0.096355 < Tau <= 0.359004
tp = 216, tn = 429, fp = 20, fn = 335
precision: 0.9152542372881356, covering: 0.39201451905626133, error: 0.044543429844097995, accuracy: 0.645, f1: 0.5489199491740788

rule: SafetyMargin > 0.070627 and Eta > 0.372908
tp = 192, tn = 441, fp = 8, fn = 359
precision: 0.96, covering: 0.3484573502722323, error: 0.017817371937639197, accuracy: 0.633, f1: 0.511318242343542

rule: 0.041272 < SafetyMargin <= 0.071004 and Tau <= 0.619041
tp = 142, tn = 434, fp = 15, fn = 409
precision: 0.9044585987261147, covering: 0.2577132486388385, error: 0.0334075723830735, accuracy: 0.576, f1: 0.4011299435028249

rule: SafetyMargin > 0.010931 and Eta > 0.748042 and Tau <= 0.682408
tp = 145, tn = 430, fp = 19, fn = 406
precision: 0.884

# Skope Rules performance

In [7]:
#mix both rules, model for collision and model for no collision.
skrules_col = pd.read_csv('simulationVictorTopological/skope/skope_rules_collisions.csv', header=None, names=["Rule","Covering","Error"])
skrules_nocol = pd.read_csv('simulationVictorTopological/skope/skope_rules_noncollisions.csv', names=["Rule","Covering","Error"])

skrules = pd.concat([skrules_col, skrules_nocol], ignore_index=True)
skrules

,Rule,Covering,Error
0,RULE 1: IF meanEntropy <= 2.6607576608657837 a...,COVERING: 0.6147426981919333,ERROR: 0.04017372421281218
1,RULE 2: IF medianEntropy <= 2.6532750129699707...,COVERING: 0.4927032661570535,ERROR: 0.023415977961432466
2,RULE 3: IF meanEntropy <= 2.6576385498046875 a...,COVERING: 0.2669014084507042,ERROR: 0.06650246305418717
3,RULE 4: IF meanEntropy > 2.663058042526245 and...,COVERING: 0.14004222378606615,ERROR: 0.49104859335038364
4,RULE 5: IF meanEntropy <= 2.6713720560073853 a...,COVERING: 0.13588110403397027,ERROR: 0.4920634920634921
5,RULE 6: IF meanEntropy <= 2.666360020637512 an...,COVERING: 0.12331679659815734,ERROR: 0.3579335793357934
6,RULE 7: IF meanEntropy <= 2.6630048751831055 a...,COVERING: 0.06983240223463687,ERROR: 0.2592592592592593
7,RULE 8: IF meanEntropy <= 2.663058042526245 an...,COVERING: 0.06619718309859154,ERROR: 0.265625
8,RULE 9: IF meanEntropy <= 2.6630786657333374 a...,COVERING: 0.056457304163726185,ERROR: 0.23809523809523814
9,RULE 10: IF meanEntropy <= 2.663058042526245 a...,COVERING: 0.03309859154929577,ERROR: 0.25396825396825395


In [8]:
skrules["Rule"][0]

'RULE 1: IF meanEntropy <= 2.6607576608657837 and medianEntropy <= 2.6585750579833984 THEN output = 1'

In [9]:
# adjust columns values
for i in range(len(skrules)+1):   
    skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("AND","and"))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("{",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("}",""))
#skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("output in ","output = "))
skrules["Covering"] = skrules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
skrules["Error"] = skrules["Error"].apply(lambda x: x.replace("ERROR: ",""))
skrules['Output'] = skrules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("IF ",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))

In [10]:
# for noncoll 
data = pd.read_csv("simulationVictorTopological/test.csv")
skrules_noncoll = skrules[skrules["Output"]==0]
preds_skope = evaluate_rules(data, skrules_noncoll, "output")

rule: meanEntropy > 2.6773265600204468 and medianEntropy > 2.692625045776367
tp = 234, tn = 26, fp = 1, fn = 343
precision: 0.9957446808510638, covering: 0.4055459272097054, error: 0.037037037037037035, accuracy: 0.4304635761589404, f1: 0.5763546798029556

rule: meanEntropy > 2.69325053691864
tp = 228, tn = 26, fp = 1, fn = 349
precision: 0.9956331877729258, covering: 0.3951473136915078, error: 0.037037037037037035, accuracy: 0.4205298013245033, f1: 0.56575682382134

rule: meanEntropy <= 2.695651054382324 and meanEntropy > 2.677270531654358
tp = 248, tn = 24, fp = 3, fn = 329
precision: 0.9880478087649402, covering: 0.42980935875216636, error: 0.1111111111111111, accuracy: 0.4503311258278146, f1: 0.5990338164251208

rule: meanEntropy <= 2.693302869796753 and meanEntropy > 2.6696172952651978 and medianEntropy > 2.675075054168701
tp = 231, tn = 22, fp = 5, fn = 346
precision: 0.9788135593220338, covering: 0.4003466204506066, error: 0.18518518518518517, accuracy: 0.41887417218543044, f1: 

In [11]:
# for coll
data = pd.read_csv("simulationVictorTopological/test.csv")
skrules_noncoll = skrules[skrules["Output"]==1]
preds_skope = evaluate_rules(data, skrules_noncoll, "output")

rule: meanEntropy <= 2.6607576608657837 and medianEntropy <= 2.6585750579833984
tp = 14, tn = 20, fp = 7, fn = 563
precision: 0.6666666666666666, covering: 0.024263431542461005, error: 0.25925925925925924, accuracy: 0.056291390728476824, f1: 0.046822742474916385

rule: medianEntropy <= 2.6532750129699707
tp = 4, tn = 22, fp = 5, fn = 573
precision: 0.4444444444444444, covering: 0.006932409012131715, error: 0.18518518518518517, accuracy: 0.04304635761589404, f1: 0.013651877133105802

rule: meanEntropy <= 2.6576385498046875 and medianEntropy > 2.646250009536743
tp = 9, tn = 22, fp = 5, fn = 568
precision: 0.6428571428571429, covering: 0.01559792027729636, error: 0.18518518518518517, accuracy: 0.05132450331125828, f1: 0.030456852791878174

rule: meanEntropy > 2.663058042526245 and medianEntropy <= 2.672224998474121
tp = 66, tn = 19, fp = 8, fn = 511
precision: 0.8918918918918919, covering: 0.11438474870017332, error: 0.2962962962962963, accuracy: 0.14072847682119205, f1: 0.202764976958525